<a href="https://colab.research.google.com/github/Tati-AG/TP1_Mineracao_de_Dados/blob/main/tratamento_de_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount ('/content/drive')

In [2]:
from collections import defaultdict
import pandas as pd
import numpy as np

In [ ]:
tsv_file = '/content/drive/MyDrive/imdb_data/title.basics.tsv'

data_types = defaultdict (lambda: str, {5:'Int64', 6:'Int64'})

title_basics = pd.read_table (tsv_file, sep = '\t', header = 0, dtype = data_types, na_values = '\\N',
                              converters = {7: lambda x: int(x) if x.isdigit() else pd.NA})

In [4]:
movie_basics = title_basics.loc[np.where ((title_basics['titleType'].values == 'movie'))]

In [5]:
genres_by_movie = movie_basics['genres'].str.split(',').explode()

In [6]:
movie_genres = pd.DataFrame ({'genre' : genres_by_movie.unique()})

In [7]:
movie_genre_bridge = pd.DataFrame ({'tconst' : movie_basics.loc[genres_by_movie.index]['tconst'].values,
                                    'genre' : genres_by_movie.values})

In [8]:
movie_basics.drop('genres', axis='columns', inplace=True)
del title_basics
del genres_by_movie

In [9]:
movie_basics.to_csv('/content/drive/My Drive/imdb_data/processed_data/movie_basics.csv', index = False)
movie_genres.to_csv('/content/drive/My Drive/imdb_data/processed_data/movie_genres.csv', index = False)
movie_genre_bridge.to_csv('/content/drive/My Drive/imdb_data/processed_data/movie_genre_bridge.csv', index = False)

In [10]:
tsv_file = '/content/drive/MyDrive/imdb_data/title.ratings.tsv'

data_types = {0:str, 1:'Float64', 2:'Int64'}

title_ratings = pd.read_table (tsv_file, sep = '\t', header = 0, dtype = data_types, na_values = '\\N')

In [11]:
movie_ratings = title_ratings.loc[np.where ((title_ratings['tconst'].isin(movie_basics['tconst'])))]

In [12]:
movie_ratings.to_csv('/content/drive/My Drive/imdb_data/processed_data/movie_ratings.csv', index = False)

In [13]:
del title_ratings

In [14]:
tsv_file = '/content/drive/MyDrive/imdb_data/title.crew.tsv'

data_types = defaultdict (lambda: str)

title_crew = pd.read_table (tsv_file, sep = '\t', header = 0, dtype = data_types, na_values = '\\N')

In [15]:
movie_crew = title_crew.loc[np.where ((title_crew['tconst'].isin(movie_basics['tconst'])))]

In [40]:
director_by_movie = movie_crew['directors'].str.split(',').explode()
movie_directors_bridge = pd.DataFrame ({'tconst' : movie_crew.loc[director_by_movie.index]['tconst'],
                                        'nconst' : director_by_movie.values}).dropna()

In [41]:
movie_directors_bridge.to_csv('/content/drive/My Drive/imdb_data/processed_data/movie_director_bridge.csv', index = False)

In [ ]:
del title_crew
del movie_crew
del director_by_movie

In [42]:
tsv_file = '/content/drive/MyDrive/imdb_data/name.basics.tsv'

data_types = defaultdict (lambda: str, {2:'Int64', 3:'Int64'})

name_basics = pd.read_table (tsv_file, sep = '\t', header = 0, dtype = data_types, na_values = '\\N')

In [43]:
movie_directors = name_basics.loc[np.where ((name_basics['nconst'].isin(movie_directors_bridge['nconst'].unique())))]

In [44]:
movie_directors.to_csv('/content/drive/My Drive/imdb_data/processed_data/movie_directors.csv', index = False)

In [ ]:
del name_basics